In [1]:
import pandas as pd

In [2]:
df_silver = pd.read_csv('data/silver/dados_limpos.csv')
df_silver['data_postagem'] = pd.to_datetime(df_silver['data_postagem'])
df_silver['score_sentimento'] = df_silver['score_sentimento'].astype(float) 

print(f"Dados Silver lidos: {df_silver.shape[0]} registros")

Dados Silver lidos: 1599603 registros


Métricas de Sentimento

In [3]:
# A. Extrair a data (sem hora) e a hora do post para agrupamento
df_silver['data_pura'] = df_silver['data_postagem'].dt.date
df_silver['hora'] = df_silver['data_postagem'].dt.hour

In [4]:
# B. Criar a tabela de métricas diárias
metricas_diarias = df_silver.groupby('data_pura').agg(
    # 1. Total de Posts (Volume)
    total_posts=('score_sentimento', 'count'), 
    # 2. Sentimento Médio Diário (KPI Chave)
    sentimento_medio=('score_sentimento', 'mean'),
    # 3. Sentimento Máximo e Mínimo (Para análise de picos de emoção)
    sentimento_max=('score_sentimento', 'max'),
    sentimento_min=('score_sentimento', 'min')
).reset_index()

In [5]:
# C. Renomear e formatar para o Power BI
metricas_diarias.columns = ['data_metrica', 'total_posts', 
                            'sentimento_medio', 'sentimento_max', 'sentimento_min']
metricas_diarias['data_metrica'] = pd.to_datetime(metricas_diarias['data_metrica'])

print("\nAgregação 1: Métricas Diárias")
print(metricas_diarias.head())


Agregação 1: Métricas Diárias
  data_metrica  total_posts  sentimento_medio  sentimento_max  sentimento_min
0   2009-04-06         3360          0.094747             1.0            -1.0
1   2009-04-07        17311          0.098518             1.0            -1.0
2   2009-04-17         3080          0.119936             1.0            -1.0
3   2009-04-18        21751          0.109199             1.0            -1.0
4   2009-04-19        27469          0.112542             1.0            -1.0


Volume por Hora do Dia (KPI de Engajamento)

In [6]:
# Criar a tabela de volume por hora
volume_por_hora = df_silver.groupby('hora').agg(
    total_posts=('score_sentimento', 'count'),
    sentimento_medio=('score_sentimento', 'mean')
).reset_index()

# Renomear e formatar
volume_por_hora.columns = ['hora_dia', 'total_posts', 'sentimento_medio']

print("\nAgregação 2: Volume por Hora do Dia")
print(volume_por_hora.head())


Agregação 2: Volume por Hora do Dia
   hora_dia  total_posts  sentimento_medio
0         0        80842          0.104444
1         1        75247          0.107344
2         2        73970          0.106608
3         3        74230          0.100238
4         4        76980          0.097699


Finalização e carga na camada gold

In [7]:
metricas_diarias.to_csv('data/gold/metricas_diarias_sentimento.csv', index=False)

volume_por_hora.to_csv('data/gold/volume_por_hora.csv', index=False)

print("\nDados agregados criados e salvos com sucesso na Camada Gold.")


Dados agregados criados e salvos com sucesso na Camada Gold.
